<h1>推薦系統<h1>


讀取資料

In [ ]:
# Load packages
import numpy as np
import pandas as pd

# Used for interact
from ipywidgets import interact, interact_manual, IntSlider, IntText

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/計算機概論") #更改路徑
os.getcwd() #查看當前路徑
!ls

計算機概論報告	資料.xlsx


In [ ]:
IO = '資料.xlsx' #定義路徑
data = pd.read_excel(IO)
data = data.drop([0,1,2]) #刪除註解列
data = data.reset_index(drop=True) #重製index並且不顯示在column
data

餐廳名稱  ...                         網址(google map)
0       小林阿盛  ...                                 小林阿盛壽司
1       好想吃冰  ...                                   好想吃冰
2         爭鮮  ...  https://goo.gl/maps/QU4dX23Y9MrPqCP69
3    有意思居酒食堂  ...  https://goo.gl/maps/NXfrH7rn8FbR5FQb8
4        韓庭州  ...  https://goo.gl/maps/CwKHsvQw8RJYfeMR7
..       ...  ...                                    ...
195   大學口胡椒餅  ...                 大學口胡椒餅(公館) - Google 地圖
196    小木屋鬆餅  ...                 小木屋鬆餅(台大店) - Google 地圖
197   鑫吉野烤肉飯  ...                  鑫吉野烤肉飯公館店 - Google 地圖
198      三米三  ...                      三米三小館 - Google 地圖
199   如來素食樂園  ...                     如來素食樂園 - Google 地圖

[200 rows x 10 columns]

In [ ]:
import requests
import json

In [ ]:
# 參考資料 http://passionfruittaiwan.blogspot.com/2020/05/python.html
# 



url = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-C0032-001?Authorization=CWB-F8BCEB2B-F5EF-4CB1-8859-A09955AF448F&format=JSON&locationName=臺北市' 
    
# 使用 GET 方式下載普通網頁
Data = requests.get(url)

# json.loads -> 將已編碼的Json字串符號，解為Pytho字串的數據類型
# 變成可讀取的資料以後，從一層一層的資料中取出需要的天氣資訊
Data = (json.loads(Data.text,encoding='utf-8'))['records']['location'][0]['weatherElement']

# 天氣資訊包含3個時段，每個時段間隔為12個小時，共包含未來36小時的天氣資訊
Data

# 取出Data中的第一個12小時的區間的最高溫資料，並且將資料型態轉為整數
int(Data[4]['time'][1]['parameter']['parameterName'])

NameError: ignored

In [ ]:
#使用者輸入
YourWeather = int(Data[4]['time'][1]['parameter']['parameterName'])
YourPlace = input("您想用餐的位置(請輸入選項代碼)：1 社科院後門; 2 溫州街; 3 正門; 4 都可以  ")
YourPrice = input("您用餐的預算(請輸入選項代碼)：1 100以下; 2 101-200; 3 201以上; 4 都可以  ")
YourMeal = input("您希望的主食(請輸入選項代碼)：1 飯食; 2 麵食; 3 其他; 4 都可以  ")
YourType = input("您希望的餐式(請輸入選項代碼)：1 台式/中式; 2 日式/韓式; 3 其他; 4 都可以  ")
YourSpicy = input("是否想吃辣(請輸入選項代碼)：1 不想要; 2 想要; 3 都可以  ")


您想用餐的位置(請輸入選項代碼)：1 社科院後門; 2 溫州街; 3 正門; 4 都可以  5
您用餐的預算(請輸入選項代碼)：1 100以下; 2 101-200; 3 201以上; 4 都可以  5
您希望的主食(請輸入選項代碼)：1 飯食; 2 麵食; 3 其他; 4 都可以  5
您希望的餐式(請輸入選項代碼)：1 台式/中式; 2 日式/韓式; 3 其他; 4 都可以  5
是否想吃辣(請輸入選項代碼)：1 不想要; 2 想要; 3 都可以  5


In [ ]:
#篩選推薦

if int(YourWeather) > 30:
  Rule_Weather = 2
elif int(YourWeather) < 15:
  Rule_Weather = 0
else:
  Rule_Weather = 1

Rule_Place = int(YourPlace) - 1
Rule_Price = int(YourPrice) - 1
Rule_Meal = int(YourMeal) - 1
Rule_Type = int(YourType) - 1
Rule_Spicy = int(YourSpicy) - 1

data['積分'] = 0 #首先建立新的積分列

#當氣溫符合所設定的選項時，即Rule_Weather = 0 or 1 or 2時，符合選項的餐廳會加10分到積分列中。
if (Rule_Weather == 0) or (Rule_Weather == 1) or (Rule_Weather == 2) : 
  data.loc[data['地區'] == Rule_Weather, '積分'] += 10

#當地區符合所設定的選項時，即Rule_Place = 0 or 1 or 2時，符合選項的餐廳會加10分到積分列中。
#而當使用者選擇都可以時，即Rule_Place = 3時，則全部餐廳都加10分。
if Rule_Place == 3 :
  data['積分'] += 10
elif (Rule_Place == 0) or (Rule_Place == 1) or (Rule_Place == 2) :
  data.loc[data['地區'] == Rule_Place, '積分'] += 10

#當價位符合所設定的選項時，即Rule_Price = 0 or 1 or 2時，符合選項的餐廳會加10分到積分列中。
#而當使用者選擇都可以時，即Rule_Price = 3時，則全部餐廳都加10分。
if Rule_Price == 3 :
  data['積分'] += 10
elif (Rule_Price == 0) or (Rule_Price == 1) or (Rule_Price == 2) :
  data.loc[data['價位'] == Rule_Price, '積分'] += 10

#當主食符合我們所設定的選項時，即Rule_Meal = 0 or 1 or 2時，符合選項的餐廳會加10分到積分列中。
#而當使用者選擇都可以時，即Rule_Meal = 3時，則全部餐廳都加10分。
if Rule_Meal == 3 :
  data['積分'] += 10
elif (Rule_Meal == 0) or (Rule_Meal == 1) or (Rule_Meal == 2) :
  data.loc[data['主食'] == Rule_Meal, '積分'] += 10

#當餐式符合所設定的選項時，即Rule_Type = 0 or 1 or 2時，符合選項的餐廳會加10分到積分列中。
#而當使用者選擇都可以時，即Rule_Type = 3時，則全部餐廳都加10分。
if Rule_Type == 3 :
  data['積分'] += 10
elif (Rule_Type == 0) or (Rule_Type == 1) or (Rule_Type == 2) :
  data.loc[data['餐式'] == Rule_Type, '積分'] += 10

#當辛香辣符合所設定的選項時，即Rule_Spicy = 0 or 1時，符合選項的餐廳會加10分到積分列中。
#而當使用者選擇都可以時，即Rule_Spicy = 2時，則全部餐廳都加10分。
if Rule_Spicy == 2 :
  data['積分'] += 10
elif (Rule_Spicy == 0) or (Rule_Spicy == 1) :
  data.loc[data['辛香辣'] == Rule_Spicy, '積分'] += 10

#最後來觀察一下積分總和後的新表單。
data

餐廳名稱 （餐點名稱）  ...                         網址(google map)  積分
0       小林阿盛    烏龍麵  ...                                 小林阿盛壽司  10
1       好想吃冰     咖哩  ...                                   好想吃冰  10
2         爭鮮   迴轉壽司  ...  https://goo.gl/maps/QU4dX23Y9MrPqCP69  10
3    有意思居酒食堂     串燒  ...  https://goo.gl/maps/NXfrH7rn8FbR5FQb8  10
4        韓庭州   石鍋拌飯  ...  https://goo.gl/maps/CwKHsvQw8RJYfeMR7  10
..       ...    ...  ...                                    ...  ..
195   大學口胡椒餅    NaN  ...                 大學口胡椒餅(公館) - Google 地圖   0
196    小木屋鬆餅    NaN  ...                 小木屋鬆餅(台大店) - Google 地圖   0
197   鑫吉野烤肉飯    NaN  ...                  鑫吉野烤肉飯公館店 - Google 地圖   0
198      三米三    NaN  ...                      三米三小館 - Google 地圖   0
199   如來素食樂園    NaN  ...                     如來素食樂園 - Google 地圖   0

[200 rows x 11 columns]

In [ ]:
#將餐廳依照積分數值由最高到最低排序。
data_random = data.iloc[np.random.permutation(data.index)].reset_index(drop=True)
newdata = data_random.sort_values('積分', ascending = False)

newdata

餐廳名稱    （餐點名稱）  ...                         網址(google map)  積分
145         柒食貳     現炸雞排飯  ...  https://goo.gl/maps/XfLios9YYjkLbGLf7  10
159         溏老鴨       小火鍋  ...  https://goo.gl/maps/fUc8eQifZtUdyGtd8  10
96         麥子磨麵   番茄豬肉片拉麵  ...  https://goo.gl/maps/X3o2SnTkqQmztUMN8  10
75           蠶居     冰糖豬腳飯  ...  https://goo.gl/maps/GnaqLYH7F69a3CK79  10
53         三個傻瓜  花椰菜馬鈴薯咖哩  ...        https://g.page/Currytoast?share  10
..          ...       ...  ...                                    ...  ..
76          頂呱呱       NaN  ...                        頂呱呱 - Google 地圖   0
78        貞有緣素食       NaN  ...  https://goo.gl/maps/EWRhx2HkPuLV8Wqv7   0
79      阿孟石碗燒河粉     石碗燒河粉  ...  https://goo.gl/maps/YLCHZF3STx5GACQt7   0
80           大埔       NaN  ...                  大埔鐵板燒 公館店 - Google 地圖   0
199  Ville café       NaN  ...                       義樂麵屋 - Google 地圖   0

[200 rows x 11 columns]

In [ ]:
#選擇方案1:直接取表單中積分排序最前面的5家
if len(data.axes[0]) > 4 :
  the_top_five = newdata.head(5)
else:
  print("No data")

#選擇方案2:先取出表單中積分排序最前面的10家再隨機抽選5間
#if len(data.axes[0]) > 4 :
#  the_top_ten = newdata.head(10)
#  the_top_five = the_top_ten.sample(5)
#else:
#  print("No data")


In [ ]:
#最後得出前五個推薦結果。
the_top_five

餐廳名稱    （餐點名稱）  ...                         網址(google map)  積分
145   柒食貳     現炸雞排飯  ...  https://goo.gl/maps/XfLios9YYjkLbGLf7  10
159   溏老鴨       小火鍋  ...  https://goo.gl/maps/fUc8eQifZtUdyGtd8  10
96   麥子磨麵   番茄豬肉片拉麵  ...  https://goo.gl/maps/X3o2SnTkqQmztUMN8  10
75     蠶居     冰糖豬腳飯  ...  https://goo.gl/maps/GnaqLYH7F69a3CK79  10
53   三個傻瓜  花椰菜馬鈴薯咖哩  ...        https://g.page/Currytoast?share  10

[5 rows x 11 columns]